# Ukol 01 - Build GLM model to predict attrition of employee (team work)
* split data - test, train, validation
* train GLM (logistic regression) model to predict attrition of given employe on training set
* cross validate on test sample
* find the 'best' model based on precision
* compute precision on validate sample
* paste your result into Slack - lecture_05
* you can paste working results too to made other teams to work hard

In [1]:
import numpy as np
import pandas as pd  # library when dealing with tabular data
import os  # standard set of OS-tools
import sys

import psycopg2        # for connection to PostgreSQL DB
import psycopg2.extras # for Dictionary-like cursor

from matplotlib import pyplot
import matplotlib.pyplot as plt

In [2]:
print(os.getcwd())

C:\Adam\School_FJFI\magisterske\3\DAS\DAS-ukoly\ukol_01


## Connection to DB
Our database is PostgreSQL on kmlinux.fjfi.cvut.cz.
The IP addresses from which the database can be accessed are limited to the FJFI range. If you want to access the database from other IPs, you must create a tunnel. Use VPN https://nms.fjfi.cvut.cz/wiki/Openvpn.fjfi.cvut.cz

In [27]:
password = open('secret.txt').read().strip()
params = {
'host': 'kmlinux.fjfi.cvut.cz',
'user': 'francji1',
'password': password,
'dbname': 'francji1_das'
}
conn = psycopg2.connect(**params)
conn

<connection object at 0x000001D44C461BF8; dsn: 'user=francji1 password=xxx dbname=francji1_das host=kmlinux.fjfi.cvut.cz', closed: 0>

In [ ]:
empData = pd.read_sql_query("SELECT * FROM attrition", conn)

In [24]:
empData.to_csv("empdata.csv", encoding='utf-8', index=False)

In [25]:
empData = pd.read_csv("empdata.csv")

In [26]:
empData.head(5)

,Unnamed: 0,age,attrition,business_travel,daily_rate,department,distance_from_home,education,education_field,employee_count,...,relationship_satisfaction,standard_hours,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_in_current_role,years_since_last_promotion,years_with_curr_manager
0,0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,...,1,80,0,8,0,1,6,4,0,5
1,1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,...,4,80,1,10,3,3,10,7,1,7
2,2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,...,2,80,0,7,3,3,0,0,0,0
3,3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,...,3,80,0,8,3,3,8,7,3,0
4,4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,...,4,80,1,6,3,3,2,2,2,2


In [6]:
print(empData.shape)

(1470, 35)


In [7]:
print(empData.columns)

Index(['age', 'attrition', 'business_travel', 'daily_rate', 'department',
       'distance_from_home', 'education', 'education_field', 'employee_count',
       'employee_number', 'environment_satisfaction', 'gender', 'hourly_rate',
       'job_involvement', 'job_level', 'job_role', 'job_satisfaction',
       'marital_status', 'monthly_income', 'monthly_rate',
       'num_companies_worked', 'over18', 'over_time', 'percent_salary_hike',
       'performance_rating', 'relationship_satisfaction', 'standard_hours',
       'stock_option_level', 'total_working_years', 'training_times_last_year',
       'work_life_balance', 'years_at_company', 'years_in_current_role',
       'years_since_last_promotion', 'years_with_curr_manager'],
      dtype='object')


In [8]:
empData['attrition'] = empData['attrition'].map({'Yes': 1, 'No': 0})

In [9]:
from sklearn.model_selection import train_test_split
shuffled = empData.sample(frac=1, random_state=42)
data_train, data_test = train_test_split(shuffled, test_size=0.2, random_state=42)

In [10]:
X_train = data_train.loc[:, empData.columns != 'attrition']
y_train = data_train['attrition']
X_test = data_test.loc[:, empData.columns != 'attrition']
y_test = data_test['attrition']

In [11]:
features = ['age','years_at_company','education'] #init, should be definitely changed

## Logistic regression

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [13]:
lgr = LogisticRegression()
lgr.fit(X_train[features], y_train)

c:\users\admin\.conda\envs\test_venv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
y_test_pred = pd.Series(lgr.predict(X_test[features]), name = 'prediction')
acc_test = accuracy_score(y_test_pred, y_test)
print("Accuracy is:", acc_test)

Accuracy is: 0.8333333333333334


In [15]:
#kfold = cross_val_score(lgr, X_train[features], y_train, cv=2)
#print('5-fold cross-validation results:' , kfold)